In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from airfoil import (
    Airfoil
)
from airfoil.wing import (
    angle_degrees_to_slope,
    mirror,
    auto_piecewise,
    auto_interpolate,
    ellipse_quadrant,
    calculated_wing_cube_loading,
    calculate_wing_area,
)
from airfoil.util.linestring_helpers import resample_shapes

In [ ]:
leading_edge = mirror(auto_piecewise([
    (25, lambda x: 0),
    (300, lambda x: -np.tan(np.deg2rad(5))*x),
    (600, lambda x: -np.tan(np.deg2rad(3))*x),
    (700, lambda x: -np.tan(np.deg2rad(3))*x+ellipse_quadrant(100,50,x))
]))
trailing_edge = mirror(auto_piecewise([
    (25, lambda x: -100),
    (150, lambda x: np.tan(np.deg2rad(5))*x-5),
    (650, lambda x: np.tan(np.deg2rad(-2))*x),
    (700, lambda x: np.tan(np.deg2rad(-2))*x-ellipse_quadrant(50,10,x))
]))
chord = lambda x: leading_edge(x)-trailing_edge(x)
dihedral = mirror(auto_piecewise([
    (25, lambda _: 0),
    (500, lambda x: angle_degrees_to_slope(1)*x),
    (700, lambda x: angle_degrees_to_slope(15)*x),
]))
thickness = mirror(auto_interpolate([
    [  0, 0.15],
    [150, 0.15],
    [600, 0.09],
    [700, 0.15],
]))
upper_surface = lambda x: dihedral(x)+thickness(x)/2 * chord(x)
lower_surface = lambda x: dihedral(x)-thickness(x)/2 * chord(x)

washout = mirror(auto_interpolate([
    [  0, 3],
    [150, 3],
    [400, 0],
    [700, 0],
]))

In [ ]:
xy = pd.read_csv("./data/fx63137-il.csv", dtype=pd.Float64Dtype())

shape_a = xy.values.astype(np.float32)
shape_b = Airfoil.from_naca_designation("2412",100).points
a1,b1 = resample_shapes([
    shape_a,
    shape_b,
], deflection_angle_split_deg=90,)

airfoil_shape = lambda x: x/700 * (b1-a1)+a1
ax2:plt.Axes
fig,(ax,ax2) = plt.subplots(2)
for xx in np.linspace(0,700,5):
    shape = airfoil_shape(xx)
    ax.plot(*shape.T)
ax2.plot(*a1.T,"o",markersize=2)
ax2.plot(*b1.T,"o",markersize=2)
for aa,bb in zip(a1,b1):
    ax2.add_line(plt.Line2D([aa[0],bb[0]],[aa[1],bb[1]]))

In [ ]:
fig, (ax1,ax2, ax3) = plt.subplots(3,1, figsize=(15,4))
x = np.linspace(-700,700,600)
y1 = np.array([leading_edge(xi) for xi in x])
y2 = np.array([trailing_edge(xi) for xi in x])
ax1.plot(x,y1)
ax1.plot(x,y2)
ax1.set_aspect("equal")

ax2.plot(x, np.array([upper_surface(xi) for xi in x]))
ax2.plot(x, np.array([lower_surface(xi) for xi in x]))
ax2.set_aspect("equal")

ax3.plot(x, np.array([washout(xi) for xi in x]))
#ax3.set_aspect("equal")

In [ ]:
wing_area = calculate_wing_area(x, y1,y2)
wing_area_m2 = wing_area/1000**2
wing_span = np.max(x)-np.min(x)
mean_chord = wing_area/wing_span
aspect_ratio = wing_span/mean_chord

mass_estimate = 1.5
wing_cube_loading = calculated_wing_cube_loading(1.5,wing_area_m2)

print(f"""
{wing_span=:.1f}
{mean_chord=:.1f}
{aspect_ratio=:.1f}
{wing_area_m2=:.3f}
{mass_estimate=:.1f} kg
{wing_cube_loading=:.1f}
""")

In [ ]:

from typing import Callable
def create_airfoil_sampler(
    points          : Callable[[float],np.ndarray],
    leading_edge    : Callable[[float],float],
    dihedral        : Callable[[float],float],
    chord           : Callable[[float],float],
    washout         : Callable[[float],float],
    rotation_center : Callable[[float],float],

)->Airfoil:
    return lambda x: (
        Airfoil(points(x)/100*chord(x))
        .with_translation((-rotation_center(x),0))
        .with_rotation(washout(x))
        .with_translation((rotation_center(x),0))
        .with_translation((-leading_edge(x), dihedral(x)))
    )

In [ ]:
afs = create_airfoil_sampler(
    points = airfoil_shape,
    leading_edge=leading_edge,
    dihedral=dihedral,
    chord=chord,
    washout=washout,
    rotation_center=lambda x: chord(x)*0.25
)

In [ ]:

fig,ax =plt.subplots(figsize=(20,10))
for x in np.linspace(0,690,15):
    shp = afs(x)
    shp.plot_raw(ax=ax)